# Example 1

In [1]:
import ee
import eerepr
ee.Initialize()
from geeo import run_param, VisMap

In [2]:
M = VisMap()
M.show()

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [3]:
# specify derised parameters (non specified parameters will be set to default values from blueprint)
prm = {'SENSORS': ['L5', 'L7', 'L8', 'L9'], 'EXCLUDE_SLCOFF': False, 'ROI': [-60.365753, -18.324544, -59.876862, -18.002244],
       'YEAR_MIN': 2023, 'YEAR_MAX': 2023, 'MONTH_MIN': 1, 'MONTH_MAX': 12, 'MAX_CLOUD': 100,
       'FEATURES': ['BLU', 'GRN', 'RED', 'NIR', 'SW1', 'SW2', 'NDVI', 'NBR', 'TCG', 'TCB', 'TCW', 'NDWI', 'KNDVI'],
       'MASKS_LANDSAT': ['cloud', 'cshadow', 'fill', 'dilated'], 'MASKS_LANDSAT_CONF': 'Medium',
       'TSM': True,  
       'TSI': None,
       'FOLD_CUSTOM': {'month': ['1-4', '5-8', '9-12']}, 'FOLD_YEAR': False,
       'STM': ['p10', 'p50', 'p90', 'stdDev'], 'STM_BASE_IMGCOL': 'TSS', 'STM_FOLDING': False, 'EXPORT_STM': False,
       'PBC': 'NLCD', 'PBC_INVERT_QUALITY_METRIC': False}

# run processing
level3 = run_param(prm)

# get imgcols
tss = level3.get('TSS')  # Time-Series-Stack
tsm = level3.get('TSM')  # Time-Series-Mosaic
pbc = level3.get('PBC')  # Pixel-based Composites
stm = level3.get('STM')  # Spectral-Temporal-Metrics
cso = ee.Image(tsm.select('BLU').count())  # Clear-Sky-Observations based on original data (TSM)

## STM

In [ ]:
n_img = 1  # starts at 0
bands = ['RED_p50', 'GRN_p50', 'BLU_p50']

M = VisMap()
for n_img in range(0, 3):
    if isinstance(stm, ee.ImageCollection):
        img = ee.Image(stm.select(bands).toList(stm.size()).get(n_img))
    else:
        img = ee.Image(stm.select(bands))
    name = img.get('system:index').getInfo()
    vis = {'min': 0.02, 'max': 0.18}
    M.add(img, roi=level3.get('ROI'), vis_params=vis, name=name)
M.show()

## PBC

In [ ]:
bands = ['RED', 'GRN', 'BLU']

M = VisMap()
for n_img in range(0, 3):
    img = ee.Image(pbc.select(bands).toList(pbc.size()).get(n_img))
    name = img.get('system:index').getInfo()
    vis = {'min': 0.02, 'max': 0.18}
    M.add(img, roi=level3.get('ROI'), vis_params=vis, name=name)
M.show()

## TSM

In [4]:
bands = ['RED', 'GRN', 'BLU']

M = VisMap()
for n_img in range(0, 3):
    img = tsm.select(bands).first()
    name = 'TSM'
    vis = {'min': 0.02, 'max': 0.18}
    M.add(img, roi=level3.get('ROI'), vis_params=vis, name=name)
M.show()

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…